In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import pickle

In [3]:

 
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
 
offset = 20
imgSize = 300
 
folder = "Data_sign\call me"
counter = 0
hand_tracking_data=[]
 
while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture frame from camera!")
        break
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
 
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
 
        if imgCrop.size == 0:
            print("Error: cropped image is empty!")
            continue
 
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
 
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            try:
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            except Exception as e:
                print("Error occurred when resizing image:", e)
                continue
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
 
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            try:
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            except Exception as e:
                print("Error occurred when resizing image:", e)
                continue
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
        hand_tracking_data.append(hand['lmList'])
 

        cv2.imshow("ImageWhite", imgWhite)
 

    key = cv2.waitKey(1)
    if key == ord("s"):
        counter += 1
        image_file_path=f'{folder}/Image_{time.time()}.jpg'
        hand_tracking_file_path=f'{folder}/HandTrackingData_{time.time()}.pkl'
        with open(hand_tracking_file_path,'wb') as f:
            pickle.dump(hand_tracking_data,f)
        hand_tracking_data=[]
        
        cv2.imwrite(image_file_path,imgWhite)
        print(counter)
    elif key  & 0xFF == ord("q"):
         break
   
cap.release()
cv2.destroyAllWindows()


Error: cropped image is empty!
1
2
3
